In [1]:
%load_ext autoreload

%autoreload 2

In [40]:
from IPython.display import Audio


# Import Modules

In [3]:
from services import get_credentials, get_database, get_watson_service

In [94]:
vcap = get_credentials()

In [6]:
from watson_developer_cloud import LanguageTranslationV2, TextToSpeechV1, NaturalLanguageUnderstandingV1

# Database

In [12]:
db, client = get_database(vcap, 'test')

In [14]:
db.create_document({'name' : 'Manolo'})
db.create_document({'name' : 'Pedro'})

{'_id': '453a7dd2a1b9a9f088ac30ca9948a0ce',
 '_rev': '1-3ff6cb90ca13159d9caa69e42917573c',
 'name': 'Pedro'}

In [15]:
list(map(lambda doc: doc['name'], db))

['Pedro', 'Manolo']

In [16]:
db.delete()

# Translator

In [121]:
translator = get_watson_service(vcap, 'language_translator')

In [122]:
translator.get_identifiable_languages()

{'languages': [{'language': 'af', 'name': 'Afrikaans'},
  {'language': 'ar', 'name': 'Arabic'},
  {'language': 'az', 'name': 'Azerbaijani'},
  {'language': 'ba', 'name': 'Bashkir'},
  {'language': 'be', 'name': 'Belarusian'},
  {'language': 'bg', 'name': 'Bulgarian'},
  {'language': 'bn', 'name': 'Bengali'},
  {'language': 'bs', 'name': 'Bosnian'},
  {'language': 'cs', 'name': 'Czech'},
  {'language': 'cv', 'name': 'Chuvash'},
  {'language': 'da', 'name': 'Danish'},
  {'language': 'de', 'name': 'German'},
  {'language': 'el', 'name': 'Greek'},
  {'language': 'en', 'name': 'English'},
  {'language': 'eo', 'name': 'Esperanto'},
  {'language': 'es', 'name': 'Spanish'},
  {'language': 'et', 'name': 'Estonian'},
  {'language': 'eu', 'name': 'Basque'},
  {'language': 'fa', 'name': 'Persian'},
  {'language': 'fi', 'name': 'Finnish'},
  {'language': 'fr', 'name': 'French'},
  {'language': 'gu', 'name': 'Gujarati'},
  {'language': 'he', 'name': 'Hebrew'},
  {'language': 'hi', 'name': 'Hindi'},


In [33]:
translator.identify('Je ne sais pas francais')['languages'][0]

{'confidence': 0.997267, 'language': 'fr'}

In [34]:
s = translator.translate('I am very happy', source='en', target='ja')
s

'とても幸せ '

In [35]:
translator.translate(s, source='ja', target='en')

'I am very happy '

In [36]:
s = translator.translate('I am on top of the world', model_id='en-es-conversational')
s

'Estoy en la cima del mundo '

In [37]:
translator.translate(s, model_id='es-en-conversational')

"I'm on top of the world "

# Text to Speech

In [38]:
text_to_speech = get_watson_service(vcap, 'text_to_speech')

In [66]:
audioEN = text_to_speech.synthesize('Hello world!', accept='audio/ogg',
voice="en-GB_KateVoice")

In [67]:
Audio(audioEN)

In [68]:
[x['name'] for x in text_to_speech.voices()['voices']]

['es-LA_SofiaVoice',
 'pt-BR_IsabelaVoice',
 'ja-JP_EmiVoice',
 'en-US_AllisonVoice',
 'fr-FR_ReneeVoice',
 'it-IT_FrancescaVoice',
 'es-ES_LauraVoice',
 'de-DE_BirgitVoice',
 'es-ES_EnriqueVoice',
 'de-DE_DieterVoice',
 'en-US_LisaVoice',
 'en-GB_KateVoice',
 'en-US_MichaelVoice',
 'es-US_SofiaVoice']

In [69]:
audioES = text_to_speech.synthesize('Hola Mundo', accept='audio/ogg',
voice="es-ES_LauraVoice")

In [70]:
Audio(audioES)

# Speech to Text

In [65]:
speech_to_text = get_watson_service(vcap, 'speech_to_text')

In [72]:
speech_to_text.recognize(
        audioEN, content_type='audio/ogg')

{'result_index': 0,
 'results': [{'alternatives': [{'confidence': 0.996,
     'transcript': 'hello world '}],
   'final': True}]}

In [73]:
speech_to_text.recognize(
        audioEN, content_type='audio/ogg', timestamps=True,
        word_confidence=True)

{'result_index': 0,
 'results': [{'alternatives': [{'confidence': 0.996,
     'timestamps': [['hello', 0.04, 0.31], ['world', 0.31, 0.85]],
     'transcript': 'hello world ',
     'word_confidence': [['hello', 1.0], ['world', 0.994]]}],
   'final': True}]}

In [78]:
[(x['language'], x['name']) for x in speech_to_text.models()['models'] ]

[('fr-FR', 'fr-FR_BroadbandModel'),
 ('en-US', 'en-US_NarrowbandModel'),
 ('pt-BR', 'pt-BR_BroadbandModel'),
 ('en-UK', 'en-UK_BroadbandModel'),
 ('zh-CN', 'zh-CN_BroadbandModel'),
 ('ja-JP', 'ja-JP_BroadbandModel'),
 ('pt-BR', 'pt-BR_NarrowbandModel'),
 ('es-ES', 'es-ES_BroadbandModel'),
 ('ar-AR', 'ar-AR_BroadbandModel'),
 ('zh-CN', 'zh-CN_NarrowbandModel'),
 ('ja-JP', 'ja-JP_NarrowbandModel'),
 ('es-ES', 'es-ES_NarrowbandModel'),
 ('en-UK', 'en-UK_NarrowbandModel'),
 ('en-US', 'en-US_BroadbandModel')]

In [123]:
speech_to_text.recognize(
        audioES, content_type='audio/ogg', model='es-ES_BroadbandModel', timestamps=True,
        word_confidence=True, )

{'result_index': 0,
 'results': [{'alternatives': [{'confidence': 0.998,
     'timestamps': [['hola', 0.03, 0.33], ['mundo', 0.33, 0.8]],
     'transcript': 'hola mundo ',
     'word_confidence': [['hola', 1.0], ['mundo', 0.996]]}],
   'final': True}]}

# Natural Language Understanding

In [127]:
import watson_developer_cloud.natural_language_understanding.features.v1 as features


In [128]:
nlu = get_watson_service(vcap, 'natural-language-understanding')

In [130]:
nlu.analyze(text='this is my experimental text.  Bruce Banner is the Hulk and Bruce Wayne is BATMAN! Superman fears not Banner, but Wayne.',
            features=[features.Entities(), features.Keywords()])

{'entities': [{'count': 3,
   'relevance': 0.915411,
   'text': 'Bruce Banner',
   'type': 'Person'},
  {'count': 1, 'relevance': 0.296395, 'text': 'Wayne', 'type': 'Person'}],
 'keywords': [{'relevance': 0.984789, 'text': 'Bruce Banner'},
  {'relevance': 0.958833, 'text': 'Bruce Wayne'},
  {'relevance': 0.853322, 'text': 'experimental text'},
  {'relevance': 0.627454, 'text': 'Hulk'},
  {'relevance': 0.619956, 'text': 'Superman'},
  {'relevance': 0.583188, 'text': 'BATMAN'}],
 'language': 'en'}

In [137]:
nlu.analyze(text='I love you', features=[features.Emotion(), features.Sentiment()], language='en')

{'emotion': {'document': {'emotion': {'anger': 0.0,
    'disgust': 0.0,
    'fear': 0.0,
    'joy': 0.9,
    'sadness': 0.0}}},
 'language': 'en',
 'sentiment': {'document': {'label': 'positive', 'score': 0.839255}}}

In [142]:
nlu.analyze(text='Lucia odia a Margarita', features=[features.Entities(), features.Sentiment()], language='es')

{'entities': [{'count': 1,
   'relevance': 0.963296,
   'text': 'Lucia',
   'type': 'Person'},
  {'count': 1, 'relevance': 0.247607, 'text': 'Margarita', 'type': 'Person'}],
 'language': 'es',
 'sentiment': {'document': {'label': 'negative', 'score': -0.924307}}}

In [146]:
text='One day, there was a ferret in the Wellness Room (at Petco, the Wellness Room is where animals who are ill or having any type of issue are kept in order to receive special attention and any medical care needed.) He was by himself and he dooked, a lot. “Dooking” is how ferrets “talk.” Cats meow. Dogs bark. Birds chirp. Ferrets dook. It almost sounds like a chicken clucking. In talking with the General Manager, we decided that the ferret was lonely, so I made it a point to go visit him in the Wellness Room whenever possible. This ferret had a skin infection and was receiving oral medications to help clear up the infection. Well, the day finally came when he finished all of his medication and was ready to join the others on the sales floor. However, I couldn’t give him up; we had bonded during our daily visits. I’d grown accustomed to him being there in the Wellness Room, waiting for me each morning; greeting me with enthusiasm that grew each day he got better. So, I bought him and named him Gizmo since his dooking reminded me of Gizmo the Gremlin. As I was getting ready to take Gizmo home, I happened to walk past the habitat on the sales floor (I’m sure I was just heading to the ladies room, yeah, that’s it). Well, low and behold, there was a single, little female ferret all by her lonesome. How could I pack up Gizmo and just walk past her and leave her by herself? Especially knowing that ferrets do better in pairs or multiples, especially when raised together or introduced correctly. Welcome Mink to the family.'

nlu.analyze(text=text, features=[features.Entities(), features.Keywords(), features.Sentiment(), 
                                 features.Emotion(), features.Relations(), features.Concepts()])

{'concepts': [{'dbpedia_resource': 'http://dbpedia.org/resource/Medicine',
   'relevance': 0.943962,
   'text': 'Medicine'},
  {'dbpedia_resource': 'http://dbpedia.org/resource/Ferret',
   'relevance': 0.90459,
   'text': 'Ferret'},
  {'dbpedia_resource': 'http://dbpedia.org/resource/Mustelidae',
   'relevance': 0.833048,
   'text': 'Mustelidae'},
  {'dbpedia_resource': 'http://dbpedia.org/resource/Health_care',
   'relevance': 0.750068,
   'text': 'Health care'},
  {'dbpedia_resource': 'http://dbpedia.org/resource/Illness',
   'relevance': 0.678776,
   'text': 'Illness'},
  {'dbpedia_resource': 'http://dbpedia.org/resource/Carnivora',
   'relevance': 0.669715,
   'text': 'Carnivora'},
  {'dbpedia_resource': 'http://dbpedia.org/resource/Bird',
   'relevance': 0.611337,
   'text': 'Bird'},
  {'dbpedia_resource': 'http://dbpedia.org/resource/Pharmaceutical_drug',
   'relevance': 0.610033,
   'text': 'Pharmaceutical drug'}],
 'emotion': {'document': {'emotion': {'anger': 0.12589,
    'dis

# Visual Recognition

In [119]:
from watson_developer_cloud import VisualRecognitionV3

In [120]:
visual_recognition = get_watson_service(vcap, 'watson_vision_combined')

In [106]:
url = "http://weknowyourdreams.com/images/volcano/volcano-05.jpg"
visual_recognition.classify(images_url=url)

{'custom_classes': 0,
 'images': [{'classifiers': [{'classes': [{'class': 'plush (fabric for toys)',
       'score': 0.791},
      {'class': 'fabric', 'score': 0.791},
      {'class': 'false face',
       'score': 0.568,
       'type_hierarchy': '/mask/false face'},
      {'class': 'mask', 'score': 0.589},
      {'class': 'piggy bank', 'score': 0.53},
      {'class': 'lemon yellow color', 'score': 0.922},
      {'class': 'pale yellow color', 'score': 0.892}],
     'classifier_id': 'default',
     'name': 'default'}],
   'resolved_url': 'http://cdn.bulbagarden.net/upload/thumb/0/0d/025Pikachu.png/250px-025Pikachu.png',
   'source_url': 'http://cdn.bulbagarden.net/upload/thumb/0/0d/025Pikachu.png/250px-025Pikachu.png'}],
 'images_processed': 1}

In [108]:
url="http://creator.keepcalmandcarryon.com/kcp-preview/NQOxtJXW"
visual_recognition.recognize_text(images_url=url)

{'images': [{'resolved_url': 'http://creator.keepcalmandcarryon.com/kcp-preview/NQOxtJXW',
   'source_url': 'http://creator.keepcalmandcarryon.com/kcp-preview/NQOxtJXW',
   'text': 'keep\ncalm\nand\ncarry\non',
   'words': [{'line_number': 0,
     'location': {'height': 41, 'left': 69, 'top': 109, 'width': 132},
     'score': 0.9887,
     'word': 'keep'},
    {'line_number': 1,
     'location': {'height': 42, 'left': 54, 'top': 164, 'width': 160},
     'score': 0.9679,
     'word': 'calm'},
    {'line_number': 2,
     'location': {'height': 17, 'left': 110, 'top': 220, 'width': 50},
     'score': 0.9712,
     'word': 'and'},
    {'line_number': 3,
     'location': {'height': 82, 'left': 34, 'top': 231, 'width': 201},
     'score': 0.5648,
     'word': 'carry'},
    {'line_number': 4,
     'location': {'height': 49, 'left': 90, 'top': 305, 'width': 89},
     'score': 0.6411,
     'word': 'on'}]}],
 'images_processed': 1}

In [111]:
url = "https://fthmb.tqn.com/TbYcG1ZT3HMQNs2APucg94XyunE=/400x0/filters:no_upscale()/about/overlyattached-5900fa4a3df78c54563e3d8b.jpg"
visual_recognition.detect_faces(images_url=url)

{'images': [{'faces': [{'age': {'max': 24, 'min': 18, 'score': 0.502411},
     'face_location': {'height': 203, 'left': 130, 'top': 104, 'width': 165},
     'gender': {'gender': 'FEMALE', 'score': 0.989013}}],
   'resolved_url': 'https://fthmb.tqn.com/TbYcG1ZT3HMQNs2APucg94XyunE=/400x0/filters:no_upscale()/about/overlyattached-5900fa4a3df78c54563e3d8b.jpg',
   'source_url': 'https://fthmb.tqn.com/TbYcG1ZT3HMQNs2APucg94XyunE=/400x0/filters:no_upscale()/about/overlyattached-5900fa4a3df78c54563e3d8b.jpg'}],
 'images_processed': 1}